In [1]:
# from BatchGD import *
import cProfile
import torch
from numpy import dot, isinf, isnan, any, sqrt, isreal, real, nan, inf
import numpy as np
import scipy as sp
import processData
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from scipy import sparse as spa
from scipy import stats
import time
from minimize import minimize
from __future__ import division

In [2]:
def loadData(filename,path="../ml-100k/"):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
        for line in f:
            (user,movieid,rating,ts)=line.split('\t')
            data.append({ "user_id": str(user), "movie_id": str(movieid)})
            y.append(float(rating))
            users.add(user)
            items.add(movieid)

    return (data, np.array(y), users, items)

(train_data, y_train, train_users, train_items) = loadData("ua.base")
(test_data, y_test, test_users, test_items) = loadData("ua.test")
v = DictVectorizer()
X_train = v.fit_transform(train_data)
X_test = v.transform(test_data)
y_train = y_train
y_train = np.array(y_train)
y_test = y_test

X_train = np.array(X_train.todense())
X_test = np.array(X_test.todense())
print(X_train.__class__, X_train.shape)
print(y_train.__class__, y_train.shape)
print(X_test.__class__, X_test.shape)
print(y_test.__class__, y_test.shape)

<class 'numpy.ndarray'> (90570, 2623)
<class 'numpy.ndarray'> (90570,)
<class 'numpy.ndarray'> (9430, 2623)
<class 'numpy.ndarray'> (9430,)


In [3]:
data_test = open("../ml-100k/ua.test", "r").read().split("\n")[:-1]
data_train = open("../ml-100k/ua.base", "r").read().split("\n")[:-1]

def loadData2(data):
    A = np.zeros( (943, 1682))
    max = -1
    for dat in data:
        # print dat
        myDat = dat.split("\t")
        myDat = [int(k) for k in myDat]
        # print myDat[0], myDat[1], myDat[2]
        if(myDat[1] > max):
            max = myDat[1]
        A[myDat[0]-1][myDat[1]-1] = myDat[2]
    return A[:, 0:max]

datasTrain = loadData2(data_train)
datasTest = loadData2(data_test)
print(datasTest.shape)

(943, 1664)


In [4]:
def predFix(datas, test, fix):
    if fix == "row":
        res = np.zeros( (datas.shape[0], test.shape[1]) )
        for i in range(datas.shape[0]):
            sum = 0.0
            count = 0.0
            for j in range(datas.shape[1]):
                if(datas[i][j] != 0):
                    sum += datas[i][j]
                    count += 1
            res[i, :].fill(sum / count)
    else:
        res = np.zeros( (datas.shape[0], test.shape[1]))
        for i in range (test.shape[1]):
            sum = 0.0
            count = 0.0
            for j in range(datas.shape[0]):
                if(datas[j][i] != 0):
                    sum += datas[j][i]
                    count += 1
            if count == 0:
                res[:,i].fill(0)
            else:
                res[:,i].fill (sum / count)
    return res

In [39]:
predUser = predFix(datasTrain,datasTest, "row")
predMovie = predFix(datasTrain,datasTest, "col")
print(predUser[2])

[ 2.90909091  2.90909091  2.90909091 ...,  2.90909091  2.90909091
  2.90909091]


In [46]:
def extractPred(preds, test, length):
    t = 0
    res = np.zeros(length)
    for i in range(test.shape[0]):
        for j in range(test.shape[1]):
            if(test[i][j] != 0):
                res[t] = preds[i][j]
                t += 1
    return res

def RMSE(y, pred):
    diff = (y - pred) ** 2
    res = np.sqrt( np.mean(diff) )
    return res

In [47]:
predUsers = extractPred(predUser, datasTest, len(y_test))
predMovies = extractPred(predMovie, datasTest, len(y_test))
print(len(predUsers))
# print(RMSE(y_test, extractPred(predUser, datasTest) ))

9430


In [48]:
print(RMSE(y_test, predUsers))

1.0431349636


In [49]:
print(RMSE(y_test, predMovies))

1.04265262564


In [50]:
print(RMSE(y_test, (predMovies + predUsers) /2 ))

0.993910632559
